In [ ]:
# 06 - topic.ipynb

##### 토픽 모델링
##### LDA 문서 컬렉션에서 주제를 찾아내는 생성 모델
- document : 여러 topics의 혼합
- topic : 여러 word의 분포
- word : 특정 topic에서 생성된 단어
- 문서들은 topic의 혼합으로 구성
    - 뉴스 기사
        - 70% 정치 20% 경제 10% 스포츠
    - 토픽은 단어 분포를 갖는다
        - 정치 {선거 : 0.2, 대통령 : 0.15, 정부 : 0.1 ...}
        - 경제 {주식 : 0.25, 금리 : 0.2, 은행 : 0.15}
- 모든 문서를 토픽의 혼합, 토픽별 단어 분포를 랜덤하게 초기화
    - 과정을 반복하면
        - 각 단어가 어떤 토픽에서 났는 지 추정
        - 각 문서의 토픽 비율을 업데이트
        - 각 토픽의 단어 분포를 업데이트

In [2]:
# 라이브러리 설치
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt

In [3]:
documents = [
    "Python 프로그래밍은 매우 강력하고 배우기 쉬워요",
    "머신러닝은 인공지능의 핵심 기술입니다",
    "자연어처리는 NLP라고도 불립니다",
    "딥러닝은 신경망을 이용한 학습 방법입니다",
    "데이터 분석은 통계학에 기반합니다",
    "파이썬으로 머신러닝 모델을 만들 수 있습니다",
    "인공지능 기술은 빠르게 발전하고 있습니다",
    "자연어처리 모델은 텍스트를 이해할 수 있어요"
]

In [4]:
# 단어 벡터화
cv = CountVectorizer(
    max_features=50,
    stop_words=['은','는','이','가','을','를','그','그리고']
)

doc_term_matrix = cv.fit_transform(documents)
feature_names = cv.get_feature_names_out()
print(f'문서 단어 행렬 : {doc_term_matrix.shape}')
print(f'단어목록 : {feature_names} 단어목록 개수 : {len(feature_names)}')

문서 단어 행렬 : (8, 36)
단어목록 : ['nlp라고도' 'python' '강력하고' '기반합니다' '기술은' '기술입니다' '데이터' '딥러닝은' '만들' '매우'
 '머신러닝' '머신러닝은' '모델은' '모델을' '발전하고' '방법입니다' '배우기' '분석은' '불립니다' '빠르게' '쉬워요'
 '신경망을' '이용한' '이해할' '인공지능' '인공지능의' '있습니다' '있어요' '자연어처리' '자연어처리는' '텍스트를'
 '통계학에' '파이썬으로' '프로그래밍은' '학습' '핵심'] 단어목록 개수 : 36


In [ ]:
# LDA 모델 생성
lda_model = LatentDirichletAllocation(
    n_components=3 # 토픽(주제) 개수
    ,random_state=  42
    ,max_iter=20
    ,learning_method='online' # batch ( 모든 데이터를 한번에 다써서 한번 학습), online(미니배치)
)
# 모델 학습
lda_output = lda_model.fit_transform(doc_term_matrix)
print(f'문서 - 주제 행렬 : {lda_output.shape}')
#

# 각 주제 별로 상위 단어 출력
def display_topic(model, feature_names, n_top_words =5):
    print(f'각 주제별 사우이 단어 ---')
    for topic_idxm, topic in enumerate(model.components_):
        # 가장 높은 가중치를 가진 단어의 인덱스 호출
        top_words_idx = topic.argsort()[-n_top_words:][::-1]
        top_words = [feature_names[i] for i in top_words_idx]
        top_weights = [topic[i] for i in top_words_idx]
        print(f'[topic{topic}]')
        for word,weight in zip(top_words,top_weights):
            print(f'{word} : {weight:.4f}')
            print()
display_topic(lda_model,feature_names, n_top_words=5)

문서 - 주세 행렬 : (8, 3)
각 주제별 사우이 단어 ---
[topic[0.38300407 0.37702435 0.37634052 0.37702966 1.31854621 1.31159533
 0.37540282 0.38207067 0.37989072 0.36592429 0.37165923 1.30887914
 1.31775912 0.37670117 1.30433248 0.37908538 0.37405174 0.37657625
 0.37955473 1.30006301 0.38374464 0.37044813 0.36978073 1.30897987
 1.31238986 1.30769969 1.3052814  1.30522693 1.30416118 0.3872354
 1.31023919 0.37600688 0.37379224 0.38215243 0.38554479 1.31254376]]
기술은 : 1.3185

모델은 : 1.3178

핵심 : 1.3125

인공지능 : 1.3124

기술입니다 : 1.3116

[topic[0.37413448 0.37614024 0.37497223 1.30655712 0.38423062 0.38868142
 1.31035943 0.37429034 1.30804094 0.38916196 1.30910415 0.37706184
 0.37692488 1.31085849 0.37352171 0.37520832 0.38458543 1.30953655
 0.38055308 0.38575759 0.37373533 0.37634603 0.38054559 0.38031358
 0.36956293 0.37627296 1.31275362 0.38136682 0.38037569 0.37938594
 0.3663937  1.30751541 1.30872303 0.39619409 0.37726879 0.38106059]]
있습니다 : 1.3128

모델을 : 1.3109

데이터 : 1.3104

분석은 : 1.3095

머신러닝 : 1.3091



In [8]:
# 각 문서의 주요 주제
print(f'각 문서의 주요 주제------------')
for doc_idx, doc in enumerate(lda_output):
    main_topic = np.argmax(doc)
    confidence = doc[main_topic]
    print(f'문서 : {doc_idx} topic : {main_topic} 비율 : {confidence:.4f}')
    print(f'원문 : {documents[doc_idx]}')
    print()

각 문서의 주요 주제------------
문서 : 0 topic : 2 비율 : 0.9036
원문 : Python 프로그래밍은 매우 강력하고 배우기 쉬워요

문서 : 1 topic : 0 비율 : 0.8650
원문 : 머신러닝은 인공지능의 핵심 기술입니다

문서 : 2 topic : 2 비율 : 0.8312
원문 : 자연어처리는 NLP라고도 불립니다

문서 : 3 topic : 2 비율 : 0.8876
원문 : 딥러닝은 신경망을 이용한 학습 방법입니다

문서 : 4 topic : 1 비율 : 0.8655
원문 : 데이터 분석은 통계학에 기반합니다

문서 : 5 topic : 1 비율 : 0.8867
원문 : 파이썬으로 머신러닝 모델을 만들 수 있습니다

문서 : 6 topic : 0 비율 : 0.8855
원문 : 인공지능 기술은 빠르게 발전하고 있습니다

문서 : 7 topic : 0 비율 : 0.8876
원문 : 자연어처리 모델은 텍스트를 이해할 수 있어요



In [12]:
# 청와대 국민청원 데이터로드
import os
import urllib.request
import ssl
url = 'https://s3.ap-northeast-2.amazonaws.com/data10902/petition/petition_sampled.csv'

# ssl 인증서 무시
ssl._create_default_https_context = ssl._create_unverified_context

urllib.request.urlretrieve(url,'./smaple.csv')

('./smaple.csv', <http.client.HTTPMessage at 0x1c5ffda7680>)

In [24]:
df = pd.read_csv("C:\\KIM\\LLM\\smaple.csv",nrows=1000)
df.head()
# ^ 한글 깨지는 경우 ^

,article_id,start,end,answered,votes,category,title,content
0,58,2017-08-19,2017-11-17,0,21,일자리,국토교통부와 한국주택협회가 행한 부당한 행위와 권력남용에 대한 내용을 청원드립니다.,안녕하세요? 존경하고 지지하는 문재인 대통령님!\n저는 성남시 분당구 정자동 주택전...
1,63,2017-08-20,2017-09-04,0,1,보건복지,살려주세요..,안녕하십니까?\n저는 올해 63세된 홀로 사는 늙은 여자입니다...\n작년 중복날 ...
2,136,2017-08-20,2017-11-18,0,4,육아/교육,고등학교 교육 내용 수준을 낮춰주시고 실용적인 내용을 담아주세요!,저는 광주에 사는 중3 학생입니다. 고등학교 가기 직전의 학년이라 어느 때보다 고등...
3,141,2017-08-20,2017-08-27,0,0,기타,한국문화에 창조적요소를 심자,안녕하십니까\n저는 92년 한국을 알게된 종국동포 입니다.\n[저는 한 중소기업에...
4,148,2017-08-20,2017-11-18,0,7,외교/통일/국방,다문화정책 및 할랄 인증 제도,대한민국과 국민을 위해 밤낮 없이 수고하시는 대통령을 비롯한 위정자 분들께\n대한민...


In [25]:
df.shape

(1000, 8)

In [26]:
# 한국어 전처리
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
okt = Okt()

# 명사만 ㅡ 그리고 한글자 이상만
def tokenizer(doc):
    return [ token for token in okt.nouns(doc) if len(token) > 1 ]

cv = CountVectorizer(
    max_features=10000,
    stop_words=['은','는','이','가','을','를','그','그리고'],
    min_df=5,
    max_df=0.5,
    tokenizer=tokenizer
)

pet_cv = cv.fit_transform(df.content)
print(f'countvectoize 형태 : {pet_cv.shape}')

c:\Users\Playdata2\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


countvectoize 형태 : (1000, 1827)


In [27]:
len(df.category.unique())

17

In [ ]:
# LDA 모델학습
from sklearn.decomposition import LatentDirichletAllocation
import time #시간측정
lda = LatentDirichletAllocation(n_components=len(df.category.unique())
                                ,n_jobs=-1
                                ,random_state=42
                                )
pet_topics = lda.fit_transform(pet_csv)
elapsed_time = time.time() - start_time

In [ ]:
# 토픽트렌드 분석
# 시간 정보 추출
# 토픽데이터 프레임 생성
# 기존 데이터프레임에서 추출한 시간정보 추가